# Datos categóricos ordinales

Para comprender qué son los datos categóricos, remitirse al archivo `09_Categoricos_nominales` de esta misma carpeta.

En particular, los datos **categóricos ordinales** son aquellos que poseen estas características (texto extraído de ChatGPT):

Los **datos ordinales** tienen un orden o jerarquía inherente entre las categorías, pero la diferencia entre las categorías no es necesariamente igual o medible.

#### Ejemplos:
- Nivel educativo (primaria, secundaria, terciaria, universitaria)
- Clasificación en una competencia (1er lugar, 2do lugar, 3er lugar)
- Escala de satisfacción (muy insatisfecho, insatisfecho, neutral, satisfecho, muy satisfecho)

En los datos **ordinales**, existe un orden claro, pero no se puede medir la distancia exacta entre las categorías. Por ejemplo, la diferencia entre "satisfecho" y "muy satisfecho" no es necesariamente igual a la diferencia entre "neutral" y "insatisfecho".

Al igual que en el caso de los datos categóricos nominales, para que los algoritmos de análisis de datos y de aprendizaje de máquina puedan procesar estos tipos de datos, se hace necesario hacer una transformación de los categóricos ordinales a datos numéricos.

A continuación, veremos las técnicas para lograr este objetivo:

## Datos categóricos ordinales: contexto

### ¿Qué tan satisfecha está con el servicio?

* Muy insatisfecho
* Insatisfecho
* Neutral
* Satisfecho
* Muy satisfecho

### ¿Cómo calificaría la calidad de los alimentos?

* Mala
* Buena
* Muy buena
* Excelente

In [1]:
import pandas as pd 
import numpy as np

/var/folders/ph/5sq6jlj161g1_2t838crnx6c0000gp/T/ipykernel_1106/4018586642.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
categorias_servicio = ["Muy insatisfecho", "Insatisfecho", "Neutral", "Satisfecho", "Muy satisfecho"]

categorias_calidad = ["Mala", "Buena", "Muy Buena", "Excelente"]

encuesta = {
    "servicio": ["Muy insatisfecho", "Insatisfecho", "Neutral", "Satisfecho", "Muy satisfecho", "Muy insatisfecho"],

    "alimentos": ["Mala", "Buena", "Muy Buena", "Excelente", "Mala", "Buena"]
}

# 0: cliente esporadico, 1: cliente frecuente
tipo_cliente = [0, 0, 1, 1, 0, 1]

In [8]:
datos = pd.DataFrame(encuesta)
datos

,servicio,alimentos
0,Muy insatisfecho,Mala
1,Insatisfecho,Buena
2,Neutral,Muy Buena
3,Satisfecho,Excelente
4,Muy satisfecho,Mala
5,Muy insatisfecho,Buena


## Codificador ordinal

Este tipo de codificador de datos categóricos a numéricos tiene una ventaja y una desventaja:

* La ventaja es que no genera una nueva columna por cada categoría, lo que evita la llamada **maldición de la dimensionalidad** que puede complejizar la tarea del procesamiento de los datos para nuestros algoritmos. Esto es lo que hace el codificador llamado **OneHotEncoder**, que se verá más abajo.

* La desventaja es que, en función del orden que le pasemos de nuestras categorías, este algoritmo les brindará un orden numérico *equidistante*. Por ejemplo, la categoría Mala sería el 0, Buena sería el 1, Muy Buena el 2 y Excelente el 3. El problema aquí radica en que según el contexto, puede que no sea correcto asumir equidistancia entre las categorías. En nuestro ejemplo, tal vez Buena y Muy Buena no pueden considerarse "alejadas" en la misma distancia que entre Mala y Buena.

In [4]:
from sklearn.preprocessing import OrdinalEncoder

Le pasamos el parametro `categories` al constructor con la lista de las categorias ordenadas como nosotros esperamos que estén ordenadas o jerarquizadas porque si no lo pasamos, el codificador les dara un orden arbitrario

In [11]:
codificador = OrdinalEncoder(categories=[categorias_servicio, categorias_calidad])

datos_ord = pd.DataFrame(codificador.fit_transform(datos[["servicio", "alimentos"]]), columns=["servicio", "alimentos"])

print(datos_ord)
print(codificador.categories_)


   servicio  alimentos
0       0.0        0.0
1       1.0        1.0
2       2.0        2.0
3       3.0        3.0
4       4.0        0.0
5       0.0        1.0
[array(['Muy insatisfecho', 'Insatisfecho', 'Neutral', 'Satisfecho',
       'Muy satisfecho'], dtype=object), array(['Mala', 'Buena', 'Muy Buena', 'Excelente'], dtype=object)]


## Codificador OneHotEncoder

In [12]:
from sklearn.preprocessing import OneHotEncoder

Este tipo de codificador nos generará una matriz con una columna por cada categoría, y en función de las observaciones, la columna de una categoría específica tendrá un 1 en una fila si esa observación coresponde con esa categoría.

La ventaja de este codificador es que no asume una equidistancia específica entre las categorías, pero al generar una nueva columna por cada categoría, nos puede llegar a quedar un dataframe muy *ancho* y con muchas dimensiones, complejizando la lectura de nuestros datos.

De allí que el uso de la codificación ordinal o la codificación OneHotEncoder dependerá del contexto en el que necesitemos aplicarlos.

Por lo general, el OneHotEncoder se utiliza para datos nominales y la codificación ordinal, como su mismo nombre lo indica, para los ordinales.

In [17]:
codificador_one = OneHotEncoder()

#print(codificador_one.fit_transform(datos[["servicio", "alimentos"]]).toarray())
#print(np.concatenate(codificador_one.categories_))

datos_one = pd.DataFrame(codificador_one.fit_transform(datos[["servicio", "alimentos"]]).toarray(),                        columns=np.concatenate(codificador_one.categories_))

datos_one

,Insatisfecho,Muy insatisfecho,Muy satisfecho,Neutral,Satisfecho,Buena,Excelente,Mala,Muy Buena
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Comparación de las codificaciones 

### Ejemplo con regresión logística

En este ejemplo, lo que nos debe importar es cómo según qué tipo de codificación usemos esto puede alterar el tipo de predicciones que realice nuestro modelo de aprendizaje de máquina. En este caso, se trata de un modelo que predice qué tipo de cliente (esporárico o frecuente) puede ser el que emita su opinión respecto a la calidad de un servicio y a la calidad de los alimentos. Dicho modelo lo entrenaremos con estos datos de "juguete" con los que estuvimos trabajando, con lo cual tengamos en cuenta que por la cantidad de dichos datos, que es muy reducida, no necesariamente sea un modelo que puede llegar a ser certero, pero a modo de ejemplo es suficiente.

Por este motivo, obviemos los detalles específicos de qué es la regresión logística.

Lo que sí podemos mencionar es que además estamos usando para este ejemplo lo que se conoce como escalamiento que es pasar los datos ordinales (ya tranformados en numéricos con la codificación ordinal) a una escala que va del 0 al 1, y no del 0 a x, como teníamos antes.

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [19]:
print("\n*** Datos Escalados de la codificación ordinal")
escalador = MinMaxScaler()
print(datos_ord)
datos_ord = escalador.fit_transform(datos_ord)
print(datos_ord)

print("\n*** Clasificación con Datos codificados con OrdinalEncoder")
modelo = LogisticRegression().fit(datos_ord, tipo_cliente) # x = datos_ord, y = tipo_cliente
print("Predicciones:", modelo.predict(datos_ord))
print("Clases correctas:", tipo_cliente)
print(modelo.predict_proba(datos_ord))

print("\n*** Clasificación con Datos codificados con OneHotEncoder")
modelo = LogisticRegression().fit(datos_one, tipo_cliente) # x = datos_ord, y = tipo_cliente
print("Predicciones:", modelo.predict(datos_one))
print("Clases correctas:", tipo_cliente)
print(modelo.predict_proba(datos_one))


*** Datos Escalados de la codificación ordinal
   servicio  alimentos
0       0.0        0.0
1       1.0        1.0
2       2.0        2.0
3       3.0        3.0
4       4.0        0.0
5       0.0        1.0
[[0.         0.        ]
 [0.25       0.33333333]
 [0.5        0.66666667]
 [0.75       1.        ]
 [1.         0.        ]
 [0.         0.33333333]]

*** Clasificación con Datos codificados con OrdinalEncoder
Predicciones: [0 0 1 1 0 0]
Clases correctas: [0, 0, 1, 1, 0, 1]
[[0.56489598 0.43510402]
 [0.50845928 0.49154072]
 [0.45180614 0.54819386]
 [0.39637512 0.60362488]
 [0.57175647 0.42824353]
 [0.50671189 0.49328811]]

*** Clasificación con Datos codificados con OneHotEncoder
Predicciones: [0 0 1 1 0 1]
Clases correctas: [0, 0, 1, 1, 0, 1]
[[0.62301432 0.37698568]
 [0.57519021 0.42480979]
 [0.32279763 0.67720237]
 [0.32279763 0.67720237]
 [0.70505958 0.29494042]
 [0.45116842 0.54883158]]
